In [41]:
import os
import chardet
import magic 
import pandas as pd
from typing import Tuple, List, Iterable

def read_raw_corpus(files: Iterable[str], corpus: str) -> Tuple[pd.DataFrame, List[str]]:
    """
    Reads all files from file and creates a dataframe containing:
        * Raw text
        * title
        * year of publication
        * period label
        * genre label
        * region label
        * corpus label (has to set as param)
    
    NOTE: The Corpus label must be either 'innsbruck' or 'manchester'.
    If it is 'manchester' another split is used to read the metadata from the filenames.
    
    Also a list of tuples containing the files which could not be processed 
    and the error which was thrown while trying to do so is returned.
    
    :param file Liste of filepaths
    :param corpus corpus label string
    :return a dataframe + a list files which could not be processed
    """
    
    data = []
    errors = []
    for file in files:
    
        m_file = magic.Magic(mime_encoding=True)
        file_encoding = m_file.from_buffer(file)
        file = file.encode(file_encoding).decode('UTF-8')
        filename = os.path.basename(file)
        genre, period, region, year, title = filename.split('_')[:5]
        if corpus.lower() == 'manchester':
            if title.endswith('RAW.txt'):
                title = title.replace('RAW.txt', '')
            else:
                title = title.replace('.txt', '')

        with open(file, 'rb') as f:
            text_bytes = f.read()
            try:
                encoding_info = chardet.detect(text_bytes)
                text = text_bytes.decode(encoding_info['encoding'])
            except Exception as e:
                errors.append((file, str(e)))
                continue

        data.append({
            'filename': file,
            'text': text,
            'title': title,
            'genre': genre,
            'period': period,
            'region': region,
            'year': year,
            'filename': filename,
            'corpus': corpus
        })
        
    df = pd.DataFrame.from_dict(data)
    df = df.set_index('filename')
    return df, errors



# Read raw innsbruck corpus files into a dataframe

In [34]:
import glob

innsbruck_raw_files = glob.glob('GermInnC/**/*.txt')
print(len(innsbruck_raw_files))

638


In [48]:
innsbruck_raw_df, errors = read_raw_corpus(innsbruck_raw_files, 'innsbruck')
print(errors)
print(innsbruck_raw_df.shape)
innsbruck_raw_df.head()

[]
(638, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
NEWS_P6_OOD_1915_Innsbrucker Nachrichten_RAW.txt,innsbruck,NEWS,P6,OOD,Aus Stadt und Land.\n\n(Trauergottesdienst.) Z...,Innsbrucker Nachrichten,1915
NEWS_P6_OMD_1928_Arbeiterstimme_RAW.txt,innsbruck,NEWS,P6,OMD,Aus Dresdens Zigaretten Industrie\r\n\tDresden...,Arbeiterstimme,1928
HUMA_P6_WMD_1920_Evangelischer Sozialismus_RAW .txt,innsbruck,HUMA,P6,WMD,Einleitung.\r\n § 1.\r\n Glaub...,Evangelischer Sozialismus,1920
DRAM_P6_WMD_1915_Heimkehr_RAW.txt,innsbruck,DRAM,P6,WMD,Zimmer im Lenz'schen Hause.\r\n(Clara steht au...,Heimkehr,1915
NEWS_P6_OOD_1914_Wiener Zeitung_RAW.txt,innsbruck,NEWS,P6,OOD,Lokales.\n(Feuer im Allgemeinen Krankenhause.)...,Wiener Zeitung,1914


In [36]:
innsbruck_raw_df.to_csv('innsbruck_dataset.csv', encoding='UTF-8')

# Read raw manchester corpus files into a dataframe

In [37]:
manchester_raw_files = glob.glob('Manchester/RAW/*.txt')

In [47]:
manchester_raw_df, errors = read_raw_corpus(manchester_raw_files, 'manchester')
print(errors)
print(manchester_raw_df.shape)
manchester_raw_df.head()

[]
(336, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
HUMA_P3_WMD_1777_HomburgRAW.txt,manchester,HUMA,P3,WMD,Nachricht von den Alterthu&#868;mern in dem Ge...,Homburg,1777
NEWS_P3_NoD_1786_wolfenbuettel1.txt,manchester,NEWS,P3,NoD,"Zeitung\r\nfür\r\nStädte, Flecken und Dörfer,\...",wolfenbuettel1,1786
NARR_P1_NoD_1658_MorgenlaendischRAW.txt,manchester,NARR,P1,NoD,Das zwey vnd dreysigste Capitel.\n Des Pr...,Morgenlaendisch,1658
SCIE_P1_WMD_1680_EpidemicaRAW.txt,manchester,SCIE,P1,WMD,Das XX. Capitel.\n Von den Schnecken.\n ...,Epidemica,1680
NEWS_P2_WMD_1701_hanau2.txt,manchester,NEWS,P2,WMD,Extraordinari Europæische Zeitung. 1701. Num. ...,hanau2,1701


In [43]:
manchester_raw_df.to_csv('manchester_dataset.csv', encoding='UTF-8')

# Concat raw dataframes

In [46]:
full_raw_df = pd.concat((manchester_raw_df, innsbruck_raw_df))
print(full_raw_df.shape)
df.head()

(974, 7)


,corpus,genre,period,region,text,title,year
filename,,,,,,,
NEWS_P6_OOD_1915_Innsbrucker Nachrichten_RAW.txt,innsbruck,NEWS,P6,OOD,Aus Stadt und Land.\n\n(Trauergottesdienst.) Z...,Innsbrucker Nachrichten,1915
NEWS_P6_OMD_1928_Arbeiterstimme_RAW.txt,innsbruck,NEWS,P6,OMD,Aus Dresdens Zigaretten Industrie\r\n\tDresden...,Arbeiterstimme,1928
HUMA_P6_WMD_1920_Evangelischer Sozialismus_RAW .txt,innsbruck,HUMA,P6,WMD,Einleitung.\r\n § 1.\r\n Glaub...,Evangelischer Sozialismus,1920
DRAM_P6_WMD_1915_Heimkehr_RAW.txt,innsbruck,DRAM,P6,WMD,Zimmer im Lenz'schen Hause.\r\n(Clara steht au...,Heimkehr,1915
NEWS_P6_OOD_1914_Wiener Zeitung_RAW.txt,innsbruck,NEWS,P6,OOD,Lokales.\n(Feuer im Allgemeinen Krankenhause.)...,Wiener Zeitung,1914


In [ ]:
full_raw_df.to_csv('full_dataset.csv', encoding='UTF-8')

In [81]:
tagged_files = glob.glob('GermInnC Tagged/**/*.txt')

In [125]:
import os
from typing import Iterable
    tagged_data = []
for file in tagged_files:
    genre, period, region, year, title = os.path.basename(file).split('_')[:5]
    tokens = []
    pos_tags = []
    lemmas = []
    with open(file, 'r', encoding='UTF-8') as f:
        for line in f:
            line = line.rstrip()
            try:
                token, pos_tag, lemma = line.split('\t')
            except ValueError:
                continue
            tokens.append(token)
            pos_tags.append(pos_tag)
            lemmas.append(lemma)

    tagged_data.append({
        'tokens': " ".join(tokens),
        'pos_tags': " ".join(pos_tags),
        'lemmas': " ".join(lemmas),
        'title': title,
        'genre': genre,
        'period': period,
        'region': region,
        'year': year,
        'corpus': None # TODO
    })

In [126]:
tagged_data[0]

{'tokens': 'Lokales und Vermischtes . Der Kaiser in Breslau . Der Kaiser , der bekanntlich heute den Feierlichkeiten anläßlich der Vermählung der Gräfin Armgard zu Stolberg = Wernigerode mit dem Grafen Platen zu Hallermund in Landeshut beiwohnen wird , begibt sich gleich nach der Trauung nach Breslau , wo er heute Abend eintrifft . Der Kaiser wird sich , wie uns ein Privat = Telegramm aus Breslau meldet , sofort nach dem fürstbischhöflichen Palais begeben , um beim Kardinal Kopp das Souper einzunehmen . Die Rückreise des Kaisers nach Berlin erfolgt am Dienstag Nachmittag . – Die Kaiserin hat ihre Teilnahme an der Hochzeitsfeierlichkeit wegen leichter Unpäßlichkeit abgesagt . Gestern Vormittag wohnte der Kaiser dem Gottesdienst zu Potsdam bei und empfing um 12 ¾ Uhr den Staatsminister Budde . – Gestern Abend begab sich der Kaiser nach dem königlichen Schauspielhaus , um der Vorstellung von Blumenthals „ Mann wir altern “ und Rofstands „ Die Romantischen “ beizuwohnen . Im Schauspielhaus

In [127]:
import pandas as pd

df = pd.DataFrame.from_dict(tagged_data)
df.to_csv('tagged_dataset.csv', encoding='UTF-8')